In [1]:
import psycopg2
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

In [2]:
# Create a cursor object
cursor = conn.cursor()


cursor.execute("SELECT count(*) FROM sites;")
results = cursor.fetchall()
num_records  = results[0][0]


In [3]:
num_records

8333

In [4]:
#Mongo db is fast, uses less memory
from pymongo import MongoClient

# set up a connection to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']


In [5]:
6964+8333

15297

In [6]:
start  = 6964
for idx in range(1,num_records+1):
    idx += start
    query = f"SELECT * FROM sites where id = {idx};"
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
#         print(row)
        doc_id = row[0]
        url = row[1]
        text = row[2]
#         print(text)
    for word in set(text.split()):
            # define the query to check if the word exists in the index
            query = {"word": word}

            # retrieve the record from the index collection
            result = db['inverted_index'].find_one(query)

            # if the word already exists in the index, add the document ID to the set
            if result:
                db['inverted_index'].update_one(query, {"$addToSet": {"doc_ids": doc_id}})
            # otherwise, create a new record for the word in the index
            else:
                db['inverted_index'].insert_one({"word": word, "doc_ids": [doc_id]})
